In [9]:
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
from binance.enums import *
apiKey, apiSecret = open('key','r').read().replace('\n','').split('|')
client = Client(apiKey, apiSecret)


In [10]:
client

In [ ]:
import decimal

info = client.get_symbol_info('BTCUSDT')
d = decimal.Decimal(str(float(info['filters'][2]['minQty'])))
d.as_tuple().exponent*-1

In [ ]:
info

In [30]:
import json
order = []
with open("./Data/history.json", "w+") as history:
    json.dump(order, history)

In [31]:
with open("./Data/history.json", "r") as history:
    data = json.load(history)


In [33]:
data == []

True

In [14]:
order = client.create_order(
    symbol='BTCUSDT',
    side=SIDE_BUY,
    type=ORDER_TYPE_LIMIT,
    timeInForce=TIME_IN_FORCE_FOK,
    quantity=0.00047,
    price='21500')

BinanceAPIException: APIError(code=-2010): Account has insufficient balance for requested action.

In [ ]:
order = client.create_order(
    symbol='BTCUSDT',
    side="SELL",
    type=ORDER_TYPE_LIMIT,
    timeInForce=TIME_IN_FORCE_FOK,
    quantity=0.00047,
    price='21300')

In [ ]:
order

In [ ]:
order["fills"][0]

In [ ]:
client.get_historical_klines("BTCUSDT", "1M", "2 months ago UTC")

In [ ]:
client.get_symbol_ticker(symbol="BTCUSDT")

In [ ]:
#all pairs
raw = client.get_margin_all_pairs()
pairs = []
for r in raw:
    pairs.append(r["symbol"])

In [ ]:
print(pairs)

In [ ]:
pairs[0]

In [13]:
order = client.create_test_order(
    symbol='BNBBTC',
    side=Client.SIDE_BUY,
    type=Client.ORDER_TYPE_MARKET,
    quantity=100)

order

{}

In [ ]:

# get market depth
depth = client.get_order_book(symbol='BNBBTC')
# place a test market buy order, to place an actual order use the create_order function
order = client.create_test_order(
    symbol='BNBBTC',
    side=Client.SIDE_BUY,
    type=Client.ORDER_TYPE_MARKET,
    quantity=100)

# get all symbol prices
prices = client.get_all_tickers()

# withdraw 100 ETH
# check docs for assumptions around withdrawals
from binance.exceptions import BinanceAPIException
try:
    result = client.withdraw(
        asset='ETH',
        address='<eth_address>',
        amount=100)
except BinanceAPIException as e:
    print(e)
else:
    print("Success")

# fetch list of withdrawals
withdraws = client.get_withdraw_history()

# fetch list of ETH withdrawals
eth_withdraws = client.get_withdraw_history(coin='ETH')

# get a deposit address for BTC
address = client.get_deposit_address(coin='BTC')

# get historical kline data from any date range

# fetch 1 minute klines for the last day up until now
klines = client.get_historical_klines("BNBBTC", Client.KLINE_INTERVAL_1MINUTE, "1 day ago UTC")

# fetch 30 minute klines for the last month of 2017
klines = client.get_historical_klines("ETHBTC", Client.KLINE_INTERVAL_30MINUTE, "1 Dec, 2017", "1 Jan, 2018")

# fetch weekly klines since it listed
klines = client.get_historical_klines("NEOBTC", Client.KLINE_INTERVAL_1WEEK, "1 Jan, 2017")

# socket manager using threads
twm = ThreadedWebsocketManager()
twm.start()

# depth cache manager using threads
dcm = ThreadedDepthCacheManager()
dcm.start()

def handle_socket_message(msg):
    print(f"message type: {msg['e']}")
    print(msg)

def handle_dcm_message(depth_cache):
    print(f"symbol {depth_cache.symbol}")
    print("top 5 bids")
    print(depth_cache.get_bids()[:5])
    print("top 5 asks")
    print(depth_cache.get_asks()[:5])
    print("last update time {}".format(depth_cache.update_time))

twm.start_kline_socket(callback=handle_socket_message, symbol='BNBBTC')

dcm.start_depth_cache(callback=handle_dcm_message, symbol='ETHBTC')

# replace with a current options symbol
options_symbol = 'BTC-210430-36000-C'
dcm.start_options_depth_cache(callback=handle_dcm_message, symbol=options_symbol)

# join the threaded managers to the main thread
twm.join()
dcm.join()